# RSA Algorithm - Network Security Project

In this Jupyter Notebook, I have implemented the **RSA encryption and decryption algorithm**, commonly used in cryptography for secure data transmission. RSA is an asymmetric cryptographic technique which uses a pair of keys: a **public key** for encryption and a **private key** for decryption.

---

## Helper functions to convert from string to decimal values and from decimal values to string

In [4]:
# Converts a string to a list of decimal numbers
def str2dec(st):
    dec_list = []
    for i in st:
        dec_list.append(ord(i))
    return dec_list

# Converts a list of decimal numbers to string
def dec2str(dec):
    str_list = []
    for i in dec:
        str_list.append(chr(i))
    return ''.join(str_list)

message = 'Hola'
print('Original String: \t', message)

dec = str2dec(message)
print('String into Decimal: \t', dec)

txt = dec2str(dec)
print('Decimal into String: \t', txt)

Original String: 	 Hola
String into Decimal: 	 [72, 111, 108, 97]
Decimal into String: 	 Hola


## Prime number generator
Generates random prime number given a value range.

In [5]:
import random
import math

def generate_prime(beg=1000, end=10000):
    beg_rand = random.randint(beg, end);
    if beg_rand % 2 == 0:
        beg_rand += 1
        
    for possiblePrime in range(beg_rand, end, 2):

        # Assume number is prime until shown it is not. 
        isPrime = True
        for num in range(3, math.floor(possiblePrime/2), 2):
            if possiblePrime % num == 0:
                isPrime = False

        if isPrime:
            return possiblePrime

The idea of RSA is based on the fact that it is difficult to factorize a large integer. The public key consists of two numbers where one number is multiplication of two large prime numbers. And private key is also derived from the same two prime numbers. So if somebody can factorize the large number, the private key is compromised. Therefore encryption strength totally lies on the key size and if we double or triple the key size, the strength of encryption increases exponentially. RSA keys can be typically 1024 or 2048 bits long, but experts believe that 1024 bit keys could be broken in the near future. But till now it seems to be an infeasible task.

## Public and Private key generation

#### The public key corresponds to the pair {e, n}

$e$ needs to be: 
* $1 < e < \phi(n)$
* coprime with $n$ and $\phi(n)$

$\phi(n) = (p - 1) * (q - 1)$ = the total number of coprimes with n

In [6]:
# This value is the multiplication of the two prime numbers,
# because the prime numbers are large this value is difficult to factorize
def generate_nkey(p, q):
    return p * q

# This 'e' key with 'n' is considered the public key
def generate_ekey(p, q):
    phi = (p-1) * (q-1)

    for e in range(random.randrange(3, phi-1, 2), phi-1):
        if math.gcd(e, phi) == 1:
            return e

#### The private key corresponds to the pair {d, n}

$d$ needs to be: 
* $d e (mod \phi(n)) = 1$

In [7]:
# This 'd' key with 'n' is considered the private key
def generate_dkey(e):
    phi = (p-1) * (q-1)

    d = int(phi / e)
    while (True):
        if (d * e) % phi == 1:
            return d
        d += 1

## Encryption and Decryption algorithm
The formula used to both encrypt and decrypt the messages is the same. All messages that are encrypted with the public key can only be decrypted with the private key, and messages encrypted with the private key can only be decrypted with the public key.

In [8]:
def endecrypt_message(m, key, n):
    
    res = 1     # Initialize result 
  
    # Update message if it is more than or equal to p 
    m = m % n  
      
    if (m == 0) : 
        return 0
  
    while (key > 0) : 
          
        # If key is odd, multiply key with result 
        if ((key & 1) == 1) : 
            res = (res * m) % n 
  
        # key must be even now 
        key = key >> 1      # key = key/2 
        m = (m * m) % n 
          
    return res

# Example

In [9]:
p = generate_prime()
q = generate_prime()

while (p == q):
    q = generate_prime()

print('Two random prime numbers')
print('\tPrime 1: ', p)
print('\tPrime 2: ', q)

n = generate_nkey(p, q)
e = generate_ekey(p, q)
d = generate_dkey(e)

print('\nn key: ', n)
print('e key: ', e)
print('d key: ', d)

print('\nPublic key {e, n}: {%d, %d}' %(e, n))
print('Private key {d, n}: {%d, %d}' %(d, n))

Two random prime numbers
	Prime 1:  9293
	Prime 2:  5021

n key:  46660153
e key:  13610883
d key:  3014267

Public key {e, n}: {13610883, 46660153}
Private key {d, n}: {3014267, 46660153}


## Encrypting with the public key and decrypting with the private key
Once the massage in encrypted using the public key, it can only be decrypted by owner of the private key, this way, the message cannot be modified during the transmition.

In [11]:
message = 'Hola, This is Tharan!'
message_dec = str2dec(message)

# Encrypt message using the public key
encrypted = [endecrypt_message(i, e, n) for i in message_dec]

# Decrypt message using the private key
decrypted = [endecrypt_message(i, d, n) for i in encrypted]

print('\nMESSAGE\n\t', message, '\n\t', message_dec)
print('\n\nENCRYPTED\n\t', encrypted)
print('\n\nDECRYPTED\n\t', dec2str(decrypted), '\n\t', decrypted)


MESSAGE
	 Hola, This is Tharan! 
	 [72, 111, 108, 97, 44, 32, 84, 104, 105, 115, 32, 105, 115, 32, 84, 104, 97, 114, 97, 110, 33]


ENCRYPTED
	 [5835246, 29833026, 13928602, 15798905, 17446309, 664742, 29768321, 38871036, 23739493, 10161232, 664742, 23739493, 10161232, 664742, 29768321, 38871036, 15798905, 45948451, 15798905, 23791601, 11918258]


DECRYPTED
	 Hola, This is Tharan! 
	 [72, 111, 108, 97, 44, 32, 84, 104, 105, 115, 32, 105, 115, 32, 84, 104, 97, 114, 97, 110, 33]


## Encrypting with the private key and decrypting with the public key
The idea here is to show that the opposit is also valid, it does not really make sense to encrypt messages using the private key, because the public key is known to everyone, this way anyone could decrypt it.
But the encryption with the public key is used to prove that the message really came from the owner of the private key.

In [13]:
message = 'Hola, This is Tharan!'
message_dec = str2dec(message)

# Encrypt message using the private key
encrypted = [endecrypt_message(i, d, n) for i in message_dec]

# Decrypt message using the public key
decrypted = [endecrypt_message(i, e, n) for i in encrypted]

print('\nMESSAGE\n\t', message, '\n\t', message_dec)
print('\n\nENCRYPTED\n\t', encrypted)
print('\n\nDECRYPTED\n\t', dec2str(decrypted), '\n\t', decrypted)


MESSAGE
	 Hola, This is Tharan! 
	 [72, 111, 108, 97, 44, 32, 84, 104, 105, 115, 32, 105, 115, 32, 84, 104, 97, 114, 97, 110, 33]


ENCRYPTED
	 [616382, 7259903, 45638308, 43138711, 20946355, 25167472, 18076985, 26246975, 24744131, 22219020, 25167472, 24744131, 22219020, 25167472, 18076985, 26246975, 43138711, 21174016, 43138711, 5076814, 13924318]


DECRYPTED
	 Hola, This is Tharan! 
	 [72, 111, 108, 97, 44, 32, 84, 104, 105, 115, 32, 105, 115, 32, 84, 104, 97, 114, 97, 110, 33]
